# Import Libraries

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

from bs4 import BeautifulSoup as bs


print('Libraries imported.')

Libraries imported.


### ######################################################

# Part 1 - Web scraping and data processing (10 marks)

In [23]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = bs(req.content,'lxml')  #BeautifulSoup
table = soup.find_all('table')
# ctrl+u to see html source of a web page

df = pd.read_html(str(table))
hoods_raw = pd.DataFrame(df[0])
hoods_raw.rename(columns={'PostalCode':'Postal Code'}, inplace=True)
hoods_raw.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [42]:
# save to excel file
# hoods_raw.to_excel('to_hoods_raw.xlsx')

In [24]:
hoods_raw.shape

(180, 3)

### "Ignore cells with a Borough that is 'Not assigned'"

In [25]:
hoods_clean = hoods_raw.loc[ hoods_raw['Borough'] != 'Not assigned' ].reset_index(drop=True)
hoods_clean.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### "If a cell has a borough but a 'Not assigned' Neighborhood, then the neighborhood will be the same as the borough"

In [41]:
# any n/a Neighborhood?
print('Number of boroughs with <Not assigned> Neighborhood = {}'.\
      format(hoods_clean.loc[ hoods_clean['Neighborhood']=='Not assigned' ]['Neighborhood'].\
             count()) )
# no

Number of boroughs with <Not assigned> Neighborhood = 0


In [39]:
# how many rows?
hoods_clean.shape[0]

103

In [43]:
# save to excel file
# hoods_clean.to_excel('to_hoods_clean.xlsx')

### ######################################################

# Part 2 - Find and add Toronto neighborhoods geographical coordinates (2 marks)

In [13]:
# import geocoder as geo

# testing a postcode
# print( geo.google('M5G, Toronto, Ontario').latlng )


# ####################################
# # initialize your variable to None
# lat_lng_coords = None

# postcode = 'M5G'

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geo.google('{}, Toronto, Ontario'.format(postcode))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]


# ####################################
# def get_geo_coords(postcode):
  
#     g = geo.google(f'{postcode}, Toronto, Ontario')
#     lat_lng_coords = g.latlng

#     latitude = lat_lng_coords[0]
#     longitude = lat_lng_coords[1]

#     return latitude
#     return longitude

# get_geo_coords('M5G')


# geocoder does not work!! 

In [14]:
# # try nominatim --> does not help for postcodes
# address = 'M5G, Toronto, Canada'  
# geolocator = Nominatim(user_agent="to_expl")
# location = geolocator.geocode(address)
# latitude = location.latitude
# longitude = location.longitude

# print(f'Lat = {latitude} , Lon = {longitude}')

### Neither geocoder nor nominatim are of help in fetching Postal Codes coordinates.
### Hence will rely on csv file provided here: http://cocl.us/Geospatial_data

In [44]:
geo_coords = pd.read_csv('http://cocl.us/Geospatial_data') #Geospatial_Coordinates.csv')
geo_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Import postcodes geo coordinates into neighborhoods dataframe

In [45]:
hoods = hoods_clean.join(geo_coords.set_index('Postal Code'),\
                               on='Postal Code' )

hoods.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [46]:
hoods.shape

(103, 5)

In [47]:
print('The dataframe has {} Boroughs and {} Postal Codes.'.format(
        len(hoods['Borough'].unique()),
        hoods['Postal Code'].unique().shape[0]
    )
)

The dataframe has 10 Boroughs and 103 Postal Codes.


In [48]:
# save to excel file
# hoods.to_excel('to_hoods+geos.xlsx')

### ######################################################

# Part 3 - Explore and cluster the neighborhoods in Toronto (3 marks)

### DISCLAIMER - PLEASE READ
--------------------------------------
### Original assignment request is 'Explore and cluster the **neighborhoods** in Toronto'
### HOWEVER, geo coordinates are only available for **postal codes**, not for neighborhoods! Also, many postal codes have multiple neighborhoods associated to them.
### Consequently, after validating this by readings in the week3 forum, ***the exploring and clustering activities here will be performed for postal codes***, not for neighborhoods.
--------------------------------------

### Use geopy library to get the latitude and longitude values of Toronto.

In [49]:
address = 'Toronto, Canada'  
geolocator = Nominatim(user_agent="to_expl")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'Toronto geo coords are: Lat = {latitude} , Lon = {longitude}')

Toronto geo coords are: Lat = 43.6534817 , Lon = -79.3839347


### Create a map of Toronto with postal codes superimposed on top.

In [50]:
# create map of Toronto using latitude and longitude values
map_toronto  = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, postcode in zip(hoods['Latitude'], hoods['Longitude'], \
                                       hoods['Borough'], hoods['Postal Code']):
    
    label = '{}, {}'.format(postcode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='yellow',  #'#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# REMOVE FOURSQUARE ID+SECRET 
# BEFORE UPLOADING TO GITHUB!

### Load personal Foursquare credentials

In [51]:
CLIENT_ID = 'ABC' #'your-client-ID' # your Foursquare ID
CLIENT_SECRET = 'XYZ' #'your-client-secret' # your Foursquare Secret
VERSION = '20180605'  #'20180604' # Foursquare API version

# print('Your credentials:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

-------------------

### Quick test of foursquare url query on a Toronto postcode

In [53]:
hood_index = 2
hood_pcode = hoods.loc[hood_index, 'Postal Code']
hood_lat = hoods.loc[hood_index, 'Latitude']
hood_long = hoods.loc[hood_index, 'Longitude']

print(f'See below some venues for Postal Code {hood_pcode}.')
print(20*'--')
    

radius = 500
limit = 10

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, hood_lat, hood_long, VERSION, radius, limit)

results = requests.get(url).json()
venues = results['response']['groups'][0]['items']

for i,v in enumerate(venues):
    print( i, v['venue']['name'], '-->',
#           v['venue']['location']['lat'],
#           v['venue']['location']['lng'],
          v['venue']['categories'][0]['name'] ) 

See below some venues for Postal Code M5A.
----------------------------------------
0 Roselle Desserts --> Bakery
1 Tandem Coffee --> Coffee Shop
2 Cooper Koo Family YMCA --> Distribution Center
3 Body Blitz Spa East --> Spa
4 Impact Kitchen --> Restaurant
5 Corktown Common --> Park
6 Dominion Pub and Kitchen --> Pub
7 Morning Glory Cafe --> Breakfast Spot
8 The Extension Room --> Gym / Fitness Center
9 The Distillery Historic District --> Historic Site


-------------------------

### Building function to collect venues info for all postcodes in Toronto

In [54]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=50):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name, end=' ')
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Code to run above function on each postal code

In [55]:
toronto_venues = getNearbyVenues(names=hoods['Postal Code'],
                                   latitudes=hoods['Latitude'],
                                   longitudes=hoods['Longitude'])

M3A M4A M5A M6A M7A M9A M1B M3B M4B M5B M6B M9B M1C M3C M4C M5C M6C M9C M1E M4E M5E M6E M1G M4G M5G M6G M1H M2H M3H M4H M5H M6H M1J M2J M3J M4J M5J M6J M1K M2K M3K M4K M5K M6K M1L M2L M3L M4L M5L M6L M9L M1M M2M M3M M4M M5M M6M M9M M1N M2N M3N M4N M5N M6N M9N M1P M2P M4P M5P M6P M9P M1R M2R M4R M5R M6R M7R M9R M1S M4S M5S M6S M1T M4T M5T M1V M4V M5V M8V M9V M1W M4W M5W M8W M9W M1X M4X M5X M8X M4Y M7Y M8Y M8Z 

----------------
----------------

### Let's explore the resulting dataframe

In [56]:
toronto_venues.shape

(1686, 7)

In [57]:
toronto_venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1686 entries, 0 to 1685
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Postcode            1686 non-null   object 
 1   Postcode Latitude   1686 non-null   float64
 2   Postcode Longitude  1686 non-null   float64
 3   Venue               1686 non-null   object 
 4   Venue Latitude      1686 non-null   float64
 5   Venue Longitude     1686 non-null   float64
 6   Venue Category      1686 non-null   object 
dtypes: float64(4), object(3)
memory usage: 72.5+ KB


In [58]:
# how many postcodes returned
print('NB number of postcodes returned is {} out of the original 103'.format(len(toronto_venues['Postcode'].unique())))

NB number of postcodes returned is 99 out of the original 103


In [59]:
# dataframe preview
toronto_venues.head()

,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,Brookbanks Pool,43.751389,-79.332184,Pool
2,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [61]:
# save to excel file
# toronto_venues.to_excel('toronto_venues.xlsx')

In [62]:
# how many unique venue categories
print('There are {} uniques venue categories'\
      .format(len( toronto_venues['Venue Category'].unique())))

There are 253 uniques venue categories


In [63]:
# top 10 most frequent venues in Toronto
top10 = toronto_venues['Venue Category'].value_counts().to_frame().head(10)
top10.index.rename('Venue Category', inplace=True)
top10.rename(columns={'Venue Category': 'Counts'}, inplace=True)
top10

,Counts
Venue Category,
Coffee Shop,123
Café,86
Restaurant,54
Pizza Place,49
Park,48
Sandwich Place,38
Bakery,34
Italian Restaurant,30
Hotel,26


In [64]:
# how many venues returned for each postocode
toronto_venues.groupby('Postcode')['Venue'].count().to_frame().transpose()

Postcode,M1B,M1C,M1E,M1G,M1H,M1J,M1K,M1L,M1M,M1N,M1P,M1R,M1S,M1T,M1V,M1W,M2H,M2J,M2K,M2N,M2P,M2R,M3A,M3B,M3C,M3H,M3J,M3K,M3L,M3M,M3N,M4A,M4B,M4C,M4E,M4G,M4H,M4J,M4K,M4L,M4M,M4N,M4P,M4R,M4S,M4T,M4V,M4W,M4X,M4Y,M5A,M5B,M5C,M5E,M5G,M5H,M5J,M5K,M5L,M5M,M5N,M5P,M5R,M5S,M5T,M5V,M5W,M5X,M6A,M6B,M6C,M6E,M6G,M6H,M6J,M6K,M6L,M6M,M6N,M6P,M6R,M6S,M7A,M7R,M7Y,M8V,M8W,M8X,M8Y,M8Z,M9A,M9C,M9L,M9M,M9N,M9P,M9R,M9V,M9W
Venue,1,2,8,3,8,1,7,9,3,4,6,8,5,13,2,12,4,50,4,35,3,4,3,5,21,21,5,3,6,4,4,5,11,8,5,34,20,3,42,17,40,4,8,18,32,2,16,4,45,50,46,50,50,50,50,50,50,50,50,26,3,4,22,35,50,17,50,50,11,5,4,4,17,15,47,22,5,5,3,24,14,39,32,12,14,15,9,2,2,16,1,9,2,1,1,8,3,10,3


-----------------
-----------------

### Many postcodes have very few venues. Let's focus the analysis only on postcodes with at least {thresh} venues.

In [66]:
thresh = 5
select_pcs = list(toronto_venues['Postcode'].value_counts()[toronto_venues['Postcode'].value_counts()>=thresh].to_frame().index)
# top_pcs

toronto_venues_select = toronto_venues.loc[ toronto_venues['Postcode'].isin(select_pcs) ]
toronto_venues_select.reset_index(drop=True).head()

,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
1,M4A,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
2,M4A,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
3,M4A,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
4,M4A,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place


In [157]:
# total number of venues drops to {?} from 1668 
print(f'Total number of venues drops to {toronto_venues_select.shape[0]} from the original 1668')

Total number of venues drops to 1595 from the starting 1668


In [159]:
# number of postcodes drops to {?} from the original 103
print('Number of postcodes drops to {} from the original 103'.\
      format(len(toronto_venues_select['Postcode'].unique())))

Number of postcodes drops to 67 from the starting 103


In [73]:
# save to excel file
# toronto_venues_select.to_excel('toronto_venues_select.xlsx')

#### Mapping the above postcodes selection

In [70]:
toronto_venues_tomap = toronto_venues_select.iloc[:,:3].drop_duplicates().reset_index(drop=True) 
toronto_venues_tomap.head()

,Postcode,Postcode Latitude,Postcode Longitude
0,M4A,43.725882,-79.315572
1,M5A,43.654260,-79.360636
2,M6A,43.718518,-79.464763
3,M7A,43.662301,-79.389494
4,M3B,43.745906,-79.352188


In [72]:
# create map of Toronto recalling latitude and longitude values
map_toronto  = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, postcode in zip(toronto_venues_tomap['Postcode Latitude'], \
                                       toronto_venues_tomap['Postcode Longitude'], \
                                       toronto_venues_tomap['Postcode']):
    
    label = '{}'.format(postcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='orange', #'#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

---------------------
----------------------

### Let's analyze each postcode

In [75]:
# one hot encoding; NB removing the default prefix which here would be 'Venue Category_'
toronto_onehot = pd.get_dummies(toronto_venues_select[['Venue Category']] ,\
                                  prefix="", prefix_sep="")

# add postcode column back to dataframe
toronto_onehot['Postcode'] = toronto_venues_select['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns] 

toronto_onehot.reset_index(drop=True).head()

,Postcode,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [77]:
toronto_onehot.shape

(1595, 243)

### Let's group rows by postcode, by taking the mean of the frequency of occurrence of each category

In [78]:
toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
toronto_grouped.head()

,Postcode,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000000,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,

In [79]:
toronto_grouped.shape

(67, 243)

In [80]:
# save to excel file
# toronto_grouped.to_excel('toronto_grouped.xlsx')

In [82]:
# toronto_grouped.dtypes
# toronto_grouped.replace(0.0, np.nan).head()

### Create dataframe showing top {?} venues for each postcode

In [136]:
# function to sort the venues in descending order.
# do not know how to modify this to account for the many postcodes with less than 10 venues.
# apparently it is not possible; see in forum Himanshu Birla reply to Efren Andres Mora 

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]


# set how many venues per postcode to be returned
# consistently with the minimum threshold number of venues per postcode set further above
num_top_venues = 5 

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))  

# create a new dataframe
postcodes_venues_sorted = pd.DataFrame(columns=columns)
postcodes_venues_sorted['Postcode'] = toronto_grouped['Postcode']

for ind in np.arange(toronto_grouped.shape[0]):
    postcodes_venues_sorted.iloc[ind, 1:] = \
        return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

postcodes_venues_sorted.reset_index(drop=True).head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1E,Rental Car Location,Bank,Mexican Restaurant,Electronics Store,Breakfast Spot
1,M1H,Fried Chicken Joint,Bakery,Bank,Athletics & Sports,Hakka Restaurant
2,M1K,Discount Store,Convenience Store,Coffee Shop,Department Store,Hobby Shop
3,M1L,Bakery,Metro Station,Soccer Field,Bus Line,Intersection
4,M1P,Indian Restaurant,Light Rail Station,Pet Store,Vietnamese Restaurant,Chinese Restaurant


In [137]:
postcodes_venues_sorted.shape

(67, 6)

In [138]:
# save to excel file
# postcodes_venues_sorted.to_excel('postcodes_venues_sorted.xlsx')

------------------
---------------

### Clustering postcodes via k-means, into a number {kclusters} of clusters

In [139]:
# note that clustering is performed on the frequencies df ('grouped'), 
# not on the ranked top {} venues df ('sorted')

# set number of clusters
kclusters = 5

# remove the 'postcodes' column
toronto_grouped_clustering = toronto_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_  #[0:10] 

array([2, 0, 2, 1, 3, 0, 2, 0, 0, 2, 2, 2, 2, 0, 2, 1, 2, 1, 1, 2, 2, 0,
       2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 2, 1, 2, 1, 2, 2, 4, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0,
       0])

In [140]:
# show size of clustered postcodes, to match total number of rows in 'toronto_grouped' df
kmeans.labels_.shape[0] #dtype

67

In [141]:
# show clusters distribution
pd.Series(kmeans.labels_).value_counts()

2    42
0    17
1     6
4     1
3     1
dtype: int64

Note that most postcodes are mainly grouped into 2 clusters only.  
Increasing the number of k-means clusters just translates into additional mini clusters made of only 1-2 postcodes.

### Create df to add cluster number label in the top {?} venues df

In [142]:
# add clustering labels
postcodes_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_) 

# import postcodes lat/long coords; rename column
toronto_merged = hoods.rename(columns={'Postal Code':'Postcode'})

# # merge toronto_grouped with toronto hoods data to add lat/long for each postcode
toronto_merged = toronto_merged.join(postcodes_venues_sorted.set_index('Postcode'), \
                                     on='Postcode', how='inner')

# NB original postcodes 103; clustered postcodes {} --> drop n/a cluster labels
# toronto_merged.dropna(subset=['Cluster Label'], inplace=True)

toronto_merged.reset_index(drop=True).head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4A,North York,Victoria Village,43.725882,-79.315572,2,Coffee Shop,Portuguese Restaurant,French Restaurant,Pizza Place,Hockey Arena
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Bakery,Pub,Park,Restaurant
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2,Furniture / Home Store,Clothing Store,Accessories Store,Boutique,Event Space
3,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Diner,Yoga Studio,Bar,Smoothie Shop
4,M3B,North York,Don Mills,43.745906,-79.352188,2,Café,Gym,Japanese Restaurant,Caribbean Restaurant,Baseball Field


In [143]:
toronto_merged.shape  #info()

(67, 11)

In [147]:
# quick check postcode by assigned cluster label
cl = 1
toronto_merged.loc[ toronto_merged['Cluster Label'] == cl , 'Postcode' ] #.count()

8     M4B
10    M6B
14    M4C
31    M6H
44    M1L
46    M3L
Name: Postcode, dtype: object

In [149]:
# save to excel file
# toronto_merged.to_excel('toronto_merged.xlsx')

### Visualize the resulting clusters

In [150]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]  
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], \
                                  toronto_merged['Postcode'], toronto_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(int(cluster)), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.5).add_to(map_clusters)
       
map_clusters

###  Examine Clusters

Cluster 0

In [152]:
cluster = 0 
toronto_merged.loc[toronto_merged['Cluster Label'] == cluster, \
                   toronto_merged.columns[[0] + list(range(6, toronto_merged.shape[1]))]]

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
26,M1H,Fried Chicken Joint,Bakery,Bank,Athletics & Sports,Hakka Restaurant
28,M3H,Bank,Coffee Shop,Pharmacy,Deli / Bodega,Supermarket
29,M4H,Indian Restaurant,Yoga Studio,Fast Food Restaurant,Supermarket,Discount Store
47,M4L,Sushi Restaurant,Gym,Pub,Fish & Chips Shop,Ice Cream Shop
55,M5M,Thai Restaurant,Coffee Shop,Sandwich Place,Restaurant,Italian Restaurant
56,M6M,Coffee Shop,Restaurant,Discount Store,Convenience Store,Sandwich Place
70,M9P,Pizza Place,Coffee Shop,Discount Store,Sandwich Place,Intersection
71,M1R,Accessories Store,Auto Garage,Middle Eastern Restaurant,Sandwich Place,Breakfast Spot
79,M4S,Sandwich Place,Dessert Shop,Coffee Shop,Gym,Italian Restaurant
82,M1T,Pizza Place,Noodle House,Italian Restaurant,Bank,Fast Food Restaurant


Cluster 1

In [153]:
cluster = 1 
toronto_merged.loc[toronto_merged['Cluster Label'] == cluster, \
                   toronto_merged.columns[[0] + list(range(6, toronto_merged.shape[1]))]]

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,M4B,Pizza Place,Bank,Fast Food Restaurant,Breakfast Spot,Gastropub
10,M6B,Bakery,Pub,Pizza Place,Japanese Restaurant,Playground
14,M4C,Skating Rink,Curling Ice,Athletics & Sports,Dance Studio,Park
31,M6H,Pharmacy,Bakery,Park,Supermarket,Café
44,M1L,Bakery,Metro Station,Soccer Field,Bus Line,Intersection
46,M3L,Grocery Store,Park,Shopping Mall,Bank,Hotel


Cluster 2

In [154]:
cluster = 2 
toronto_merged.loc[toronto_merged['Cluster Label'] == cluster, \
                   toronto_merged.columns[[0] + list(range(6, toronto_merged.shape[1]))]]

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,M4A,Coffee Shop,Portuguese Restaurant,French Restaurant,Pizza Place,Hockey Arena
2,M5A,Coffee Shop,Bakery,Pub,Park,Restaurant
3,M6A,Furniture / Home Store,Clothing Store,Accessories Store,Boutique,Event Space
4,M7A,Coffee Shop,Diner,Yoga Studio,Bar,Smoothie Shop
7,M3B,Café,Gym,Japanese Restaurant,Caribbean Restaurant,Baseball Field
9,M5B,Coffee Shop,Café,Clothing Store,Tea Room,Fast Food Restaurant
13,M3C,Restaurant,Coffee Shop,Gym,Beer Store,Bike Shop
15,M5C,Café,Cosmetics Shop,Coffee Shop,Restaurant,Gastropub
17,M9C,Café,Pet Store,Convenience Store,Pizza Place,Pharmacy
18,M1E,Rental Car Location,Bank,Mexican Restaurant,Electronics Store,Breakfast Spot


Cluster 3

In [155]:
cluster = 3 
toronto_merged.loc[toronto_merged['Cluster Label'] == cluster, \
                   toronto_merged.columns[[0] + list(range(6, toronto_merged.shape[1]))]]

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
65,M1P,Indian Restaurant,Light Rail Station,Pet Store,Vietnamese Restaurant,Chinese Restaurant


Cluster 4

In [156]:
cluster = 4 
toronto_merged.loc[toronto_merged['Cluster Label'] == cluster, \
                   toronto_merged.columns[[0] + list(range(6, toronto_merged.shape[1]))]]

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
49,M6L,Park,Trail,Bakery,Basketball Court,Construction & Landscaping


-------------
-------------

#### github link [nbviewer]  
https://nbviewer.jupyter.org/github/asaccoma/Coursera-IBM-Data-Science---Capstone-project/blob/master/Toronto%20Explorer_FINAL.ipynb